In [1]:
import numpy as np
import pandas as pd
import time
import praw
from prawcore.exceptions import NotFound,Forbidden
from textblob import TextBlob
# %pip install praw

In [66]:
def extract(lista,search_term):
    data={
        'Title':[],
        'Text':[],
        'Score':[],
        'NumberComments':[],
    }
    reddit = praw.Reddit(client_id='Z-teySKlXNxdzYpdTQr7Uw',
                        client_secret='Ze5ZS534xQMEbmtjIcOs3Mn6Vn2Wfw',
                        user_agent='frediff/1.0 by frediff')

    # Iterar sobre cada subreddit en la lista
    for subreddit_name in lista:
        nombre=subreddit_name
        print(nombre)
        subreddit = reddit.subreddit(subreddit_name)
        submissions=subreddit.search(search_term, limit=None)
        for submission in submissions:
            if submission.selftext.strip():
                data['Title'].append(submission.title)
                data['Text'].append(submission.selftext)
                data['Score'].append(submission.score)
                data['NumberComments'].append(submission.num_comments)
    return data


In [54]:
def extract_comments(subreddit_name,search_term):
    data={
        'Comments':[],
    }
    reddit = praw.Reddit(client_id='Z-teySKlXNxdzYpdTQr7Uw',
                        client_secret='Ze5ZS534xQMEbmtjIcOs3Mn6Vn2Wfw',
                        user_agent='frediff/1.0 by frediff')
    subreddit = reddit.subreddit(subreddit_name)
    submissions=subreddit.search(search_term, limit=None)
    
    comments = []   
    flag=False
    count = 0
    for submission in submissions:
        # print(submission.title)
        if submission.selftext.strip():
            submission.comments.replace_more(limit=None)
            for comment in submission.comments.list()[:100]:
                count+=1
                # print(count)
                if len(comments)>1500:
                    flag=True
                    break
                if "milei" in comment.body.lower():
                    comments.append(comment.body)
                 
        if flag:
            break
        print('time5')
        time.sleep(10)
    data['Comments']=comments
    return data

In [ ]:
datar=extract_comments('argentina','milei')

In [ ]:
datar

In [56]:
df=pd.DataFrame(datar)

In [57]:
df.shape

(1501, 1)

In [63]:
df.nunique()

Comments    1501
dtype: int64

In [74]:
df.head(10)

,Comments
0,> PD: que insoportables las focas camporistas ...
1,Lo de las jubilaciones de privilegio es para a...
2,"Como anti Milei que soy, admito:\n\n- Es loabl..."
3,Siempre dos metros adelante Milei
4,adhiero malisimo de una hora y media que duro ...
5,Milei tiro todo eso como un comentario de reed...
6,Por que Javier nos quiere meter en guerra con ...
7,Si Milei logra la transparencia aunque sea en ...
8,Che yo casi siempre encuentro la mayoría de la...
9,Declaraciones innecesarias de Milei:\n\n* El P...


In [88]:
df.tail(10)

,Comments
1491,Hola Vengo a decirte que Milei ganó el debate.
1492,Titulo alternativo: EL PELOTUDO DE MILEI LE RE...
1493,"Pecó de ego y soberbia, decía q a Massa lo gan..."
1494,OP es uno de estos kirchos que no paso compres...
1495,Milei perdió totalmente. Massa fue mucho mas e...
1496,"Es que fue así. No hay un ganador como tal, pe..."
1497,Pobre Milei. Admito que me dio algo de penita....
1498,Con este gobierno Clarin se convirtió en la iz...
1499,Lo tranquilo que me pone Milei cuando empieza ...
1500,claramente gano massa porque lo unico que hizo...


In [76]:
df.shape

(1501, 1)

In [75]:
# Crear una máscara booleana para las filas que contienen la palabra "milei" en el título o el texto
mask = df['Comments'].str.contains('milei|Milei', case=False)
# mask = ~(df['Title'].str.contains('milei|Milei', case=False) | df['Text'].str.contains('milei|Milei', case=False))

# Aplicar la máscara para filtrar las filas
df_f = df[mask]

df_f.shape

(1501, 1)

In [60]:
df_f.sample(5)

,Comments
430,Vamos 💪 Javier vos tenés huevo gigante enfrent...
998,Decir que Milei es nazi es como decir que Albe...
161,Why would you like Milei? You have actual reas...
543,Mejores jubilaciones? Baja de impuestos? En qu...
868,"Porque vivo acá, porque te das cuenta que es p..."


In [ ]:
df_f.head()
df_f.loc[117,'Text']

In [87]:
len(df_f['Title'].unique())

280

In [5]:
def fget_sents(text,method="pol",up=0.1,un=-0.1):
  if method=='pol':
    sent = TextBlob(text).sentiment.polarity
    if sent < un:
      return -1
    elif sent > up:
      return 1
    else:
      return 0
  
  elif method=='sub':
    sent = TextBlob(text).sentiment.subjectivity
    if sent > up:
      return 1
    elif sent > un:
      return 0.5
    else:
      return 0

In [13]:
df = pd.read_csv('data-NLP.csv')

In [14]:
df.shape

(1501, 1)

In [8]:
df.head()

,Comments
0,> PD: que insoportables las focas camporistas ...
1,Lo de las jubilaciones de privilegio es para a...
2,"Como anti Milei que soy, admito:\n\n- Es loabl..."
3,Siempre dos metros adelante Milei
4,adhiero malisimo de una hora y media que duro ...


In [17]:
df.rename(columns={'Comments':'coms'},inplace=True)

In [18]:
df.head()

,coms
0,> PD: que insoportables las focas camporistas ...
1,Lo de las jubilaciones de privilegio es para a...
2,"Como anti Milei que soy, admito:\n\n- Es loabl..."
3,Siempre dos metros adelante Milei
4,adhiero malisimo de una hora y media que duro ...


In [7]:
df2=df.copy()
df2['Sent']=df2['Comments'].apply(fget_sents,method='pol',up=0.1,un=-0.1)
df2.head(20)

,Comments,Sent
0,> PD: que insoportables las focas camporistas ...,-1
1,Lo de las jubilaciones de privilegio es para a...,0
2,"Como anti Milei que soy, admito:\n\n- Es loabl...",0
3,Siempre dos metros adelante Milei,0
4,adhiero malisimo de una hora y media que duro ...,0
5,Milei tiro todo eso como un comentario de reed...,0
6,Por que Javier nos quiere meter en guerra con ...,0
7,Si Milei logra la transparencia aunque sea en ...,0
8,Che yo casi siempre encuentro la mayoría de la...,0
9,Declaraciones innecesarias de Milei:\n\n* El P...,0


In [10]:
df3 = pd.read_csv('target.csv')

In [11]:
df3.shape

(1501, 1)

In [12]:
df3.head(20)

,sent
0,NEG
1,NEU
2,NEU
3,POS
4,NEG
5,NEU
6,NEG
7,POS
8,NEU
9,NEG


In [19]:
df4=pd.concat([df['coms'],df3['sent']],axis=1)

In [24]:
df4.tail()

,coms,sent
1496,"Es que fue así. No hay un ganador como tal, pe...",NEU
1497,Pobre Milei. Admito que me dio algo de penita....,NEU
1498,Con este gobierno Clarin se convirtió en la iz...,NEU
1499,Lo tranquilo que me pone Milei cuando empieza ...,NEG
1500,claramente gano massa porque lo unico que hizo...,NEU


In [25]:
df4.isna().sum()

coms    0
sent    0
dtype: int64

In [32]:
df4.to_csv('NLP-Data.csv', index=False)

In [27]:
df4['sent'].unique()

array(['NEG', 'NEU', 'POS', 'NEU '], dtype=object)

In [29]:
df4['sent'].replace('NEU ', 'NEU', inplace=True)

In [30]:
df4['sent'].unique()

array(['NEG', 'NEU', 'POS'], dtype=object)

In [31]:
df4.head()

,coms,sent
0,> PD: que insoportables las focas camporistas ...,NEG
1,Lo de las jubilaciones de privilegio es para a...,NEU
2,"Como anti Milei que soy, admito:\n\n- Es loabl...",NEU
3,Siempre dos metros adelante Milei,POS
4,adhiero malisimo de una hora y media que duro ...,NEG
